In [1]:
import os
import pickle
import numpy as np
import random
import matplotlib.image as mpimg
import cv2
from sklearn.utils import shuffle

from keras.models import Model
from keras.models import load_model
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.layers import Flatten, Dense, Dropout
from keras.regularizers import l2
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam

from pandas.io.parsers import read_csv

import copy


#
drive_info = read_csv("data/driving_log.csv", header=0, usecols=[0,1,2,3,4,5,6]).as_matrix();
modified_data = []

def perturb_angle(angle):
    new_angle = angle* (1 + np.random.uniform(-1, 1)/30.0)
    return min(1, new_angle) if new_angle > 0 else max(-1, new_angle)

def adjust_angle(angle, is_left_img):
    return angle+0.25 if is_left_img else angle-0.25

def adjust_left_angle(angle):
    return adjust_angle(angle, True)

def adjust_right_angle(angle):
    return adjust_angle(angle, False)

def get_flipped_data(line):
    new_line = copy.deepcopy(line)
    new_line[1] = -new_line[1]
    new_line[2] = True
    return new_line

for line in drive_info:
    new_line = copy.deepcopy(line)
    new_line[1] = line[3]
    modified_data.append(new_line)

    new_line = copy.deepcopy(line)
    new_line[0] = new_line[1]
    new_line[1] = adjust_left_angle(line[3])
    modified_data.append(new_line)
    
    new_line = copy.deepcopy(line)
    new_line[0] = new_line[2]
    new_line[1] = adjust_right_angle(line[3])
    modified_data.append(new_line)

# add more samples for steering angle more than 0.5 or less then -0.5

insufficient_data = [data for data in modified_data if abs(float(data[3])) > 0.5]
for line in insufficient_data:
    for i in range(0, 5):
        new_line = copy.deepcopy(line)
        new_line[1] = perturb_angle(line[3])
        modified_data.append(new_line)


modified_data = np.array(modified_data)    
print('modified data', len(modified_data))
#

#Generate data
train_data=[]
validation_data=[]

step = 0.01
per_step = 4
curent_start = 0.0

def extract_data_from_bin(current_bin):
    validation_bin = []
    
    if len(current_bin) > per_step:
        current_bin = shuffle(current_bin)
        validation_bin = current_bin[per_step:2*per_step]
        for line in validation_bin:
            validation_data.append(line)
        current_bin = current_bin[0:per_step]
    for line in current_bin:
        train_data.append(line)
        
while curent_start<1.0:
    current_bin = [data for data in modified_data if (curent_start < abs(data[1])) and (abs(data[1])<=curent_start+step)]
    extract_data_from_bin(current_bin)
    curent_start = curent_start+step
        
zero_bin = [data for data in modified_data if (data[1]==0.0)]
extract_data_from_bin(zero_bin)

train_data = np.array(train_data)
validation_data = np.array(validation_data)

print("trained data", len(train_data))
print("validation data", len(validation_data))

# prepare data to save

pickle_data = []
validation_pickle = []

for line in train_data:  
    pickle_data.append({ 'center': line[0], 'angle': line[1] })
    
for line in validation_data:  
    validation_pickle.append({ 'center': line[0], 'angle': line[1] })
    
training_file = 'train.p'
with open(training_file, 'wb') as handle:
    pickle.dump(np.array(pickle_data), handle, protocol=pickle.HIGHEST_PROTOCOL)

print("train pickle is saved")
    
validation_file = 'validation.p'
with open(validation_file, 'wb') as handle:
    pickle.dump(np.array(validation_pickle), handle, protocol=pickle.HIGHEST_PROTOCOL)

print("validation pickle is saved")
#Generate data end


#Data augmentation start

def region_of_interest(img):
    height = img.shape[0]
    width = img.shape[1]
    vertices = np.array([[(0, height-15), (0, height/2-10),
                          (width, height/2-10), (width, height-15)]],
                        dtype=np.int32)
    #defining a blank mask to start with
    mask = np.zeros_like(img)
    channel_count = img.shape[2]  # i.e. 3 or 4 depending on your image
    ignore_mask_color = (255,) * channel_count
    #filling pixels inside the polygon defined by \"vertices\" with the fill color
    cv2.fillPoly(mask, vertices, ignore_mask_color)

    #returning the image only where mask pixels are nonzero\n",
    masked_image = cv2.bitwise_and(img, mask)
    return masked_image

def rezise(img):
    return cv2.resize(img, (75,48))

def change_brightness(image):
    # Randomly select a percent change
    change_pct = random.uniform(0.3, 1.0)
    
    # Change to HSV to change the brightness V
    hsv = cv2.cvtColor(image, cv2.COLOR_RGB2HSV)
    hsv[:,:,2] = hsv[:,:,2] * change_pct
    
    #Convert back to RGB 
    img_brightness = cv2.cvtColor(hsv, cv2.COLOR_HSV2RGB)
    return img_brightness

def preprocess_image(img):
    img = region_of_interest(img)
    img = rezise(img)
    return img

#data augmentation end

FOLDER_PATH = 'data'

def read_image(image_path):
    full_image_path = os.path.join(FOLDER_PATH, image_path.strip())
    image = mpimg.imread(full_image_path)
    return preprocess_image(image)

# use flipping to avoid bias to left\right turns
# use brightness augmentaion to generalize to t2
def generate_steering_angle(data, batch_size=64):
    X = []
    Y = []
    while True:
        data = shuffle(data)    
        for line in data:
            image = read_image(line['center'])
            angle = line['angle']
            image_brightened = change_brightness(image)
            X.append(image_brightened)
            Y.append(angle)
            
            flipped_image = cv2.flip(image, 1)
            flipped_image_brightened = change_brightness(flipped_image)
            X.append(flipped_image_brightened)
            Y.append(-angle)

            if len(X)>=batch_size:
                X, Y = shuffle(X, Y)
                yield np.array(X), np.array(Y) # (image, steering angle)
                X=[]
                Y=[]
            

def generate_validation(data):
    X = []
    Y = []
    while True:
        data = shuffle(data)
        for line in data:
            angle = line['angle']
            image = read_image(line['center'])

            X.append(image)
            Y.append(angle)
            yield np.array(X), np.array(Y) # (image, steering angle)

def create_model():
    # create the base pre-trained model
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=[48, 75, 3])

    x = base_model.output
    x = Flatten()(x)

    # and a regression layer to predict steering angle
    x = Dense(1000, activation='relu', name='fc1', W_regularizer=l2(0.0001))(x)
    #x = Dropout(0.5)(x)
    x = Dense(250, activation='relu', name='fc2', W_regularizer=l2(0.0001))(x)
    x = Dropout(0.5)(x)
    predictions = Dense(1)(x)

    model = Model(input=base_model.input, output=predictions)
    # train only the top layers (which were randomly initialized)
    for layer in base_model.layers:
        layer.trainable = False
    
    model.compile(loss='mean_squared_error', optimizer='adam')
    #model = load_model("model.h5")
    return model

if __name__ == '__main__':
    model = create_model()

    training_pickle = 'train.p'
    with open(training_pickle, 'rb') as handle:
        driving_info = pickle.load(handle)

    validation_pickle = 'validation.p'
    with open(validation_pickle, 'rb') as handle:
        validation_info = pickle.load(handle)
    
    #checkpoint = ModelCheckpoint(filepath='model-{epoch:02d}.h5')
    #callback_list = [checkpoint]
    print("train size", len(driving_info))
    print("validation size", len(validation_info))

    # train the model on the new data for a few epochs
    model.fit_generator(
        generate_steering_angle(driving_info, batch_size=32),
        samples_per_epoch=768, nb_epoch=50,
        validation_data=generate_validation(validation_info),nb_val_samples=len(validation_info)/7)
        #callbacks=callback_list)

    print("Saving model weights and configuration file.")

    model.save('model.h5')
    
    print("model is saved")

Using TensorFlow backend.


modified data 24768
trained data 388
validation data 349
train pickle is saved
validation pickle is saved
train size 388
validation size 349
Epoch 1/50
768/768 [==============================] - 11s - loss: 16.5162 - val_loss: 1.4015